In [4]:
import pandas as pd
import ssl

# URL do dataset Wine
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

# Create a context that disables certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

# Ler o arquivo CSV com as colunas especificadas, definindo a coluna 'classe' como object
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

# Exibir as primeiras linhas dos dados
vinhos.head()

,classe,alcool,acido_malico,cinzas,alcalinidade_de_cinzas,magnesio,fenois_totais,flavanoides,fenois_nao_flavanoides,proantocianinas,intensidade_de_cor,matiz,od280_od315_de_vinhos_diluidos,prolina
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [5]:
# Dropar a coluna 'classe'
X = vinhos.drop('classe', axis=1)

# Reescala linear dos dados
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
# Aplicando K-Means com 4 clusters e random_state=42 para reprodutibilidade
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(X_scaled)

# Inércia do modelo
inertia = kmeans.inertia_

# Exibir o valor da inércia
print("Inércia do modelo:", inertia)


Inércia do modelo: 44.887604580852816


In [7]:
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Adicionando os rótulos dos clusters ao DataFrame
X_scaled_df['cluster'] = kmeans.labels_

# Exibindo a distribuição dos clusters
cluster_distribution = X_scaled_df['cluster'].value_counts()

# Exibindo a distribuição
print("Distribuição dos Clusters:")
print(cluster_distribution)


Distribuição dos Clusters:
cluster
2    59
1    50
0    36
3    33
Name: count, dtype: int64


In [8]:
# Adicionando a coluna 'classe' ao DataFrame para a comparação
X_scaled_df['classe'] = vinhos['classe']

# Agrupando pelos clusters e pelas classes originais
cluster_class_distribution = X_scaled_df.groupby(['classe', 'cluster']).size().unstack(fill_value=0)

# Exibindo a comparação entre as classes originais e os clusters
print("Distribuição dos Clusters por Classe Original:")
print(cluster_class_distribution)

Distribuição dos Clusters por Classe Original:
cluster   0   1   2   3
classe                 
1         2   0  57   0
2        34   2   2  33
3         0  48   0   0
